In [1]:
import torch
import torch.nn as nn

GPT_CONFIG_124M = {
    "vocab_size": 50257,    
    "context_length": 1024, 
    "emb_dim": 768,         
    "n_heads": 12,          
    "n_layers": 12,         
    "drop_rate": 0.1,       
    "qkv_bias": False       
}

In [7]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [2]:
class multiheadv2(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,attention_head,boolbias):
        super().__init__()
        self.head_dim=d_out//attention_head
        self.d_out=d_out
        self.attention_head=attention_head

        self.W_query = nn.Linear(d_in, d_out, bias=boolbias)
        self.W_key = nn.Linear(d_in, d_out, bias=boolbias)
        self.W_value = nn.Linear(d_in, d_out, bias=boolbias)

        self.out_proj = nn.Linear(d_out, d_out)  
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self,x):
        b,num_token,d_out=x.shape

        keys = self.W_key(x) 
        queries = self.W_query(x)
        values = self.W_value(x)

        keys=keys.view(b,num_token,self.attention_head,self.head_dim)
        queries=queries.view(b,num_token,self.attention_head,self.head_dim)
        values=values.view(b,num_token,self.attention_head,self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_score=queries @ keys.transpose(2,3)

        mask_bool = self.mask.bool()[:num_token, :num_token]
        attn_score.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_score / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)


        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        context_vec = context_vec.contiguous().view(b, num_token, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec
            


In [3]:
class LayerNorm(nn.Module):
    def __init__(self,emb_dim):
        super().__init__()
        self.eps=1e-5
        self.scale_params=nn.Parameter(torch.ones(emb_dim))
        self.shift_params=nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self,x):
        mean=x.mean(dim=-1,keepdim=True)
        var=x.var(dim=-1,keepdim=True,unbiased=False)
        norm=(x-mean)/torch.sqrt(var+self.eps)
        return norm*self.scale_params+ self.shift_params
    
        

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) *(x + 0.044715 * torch.pow(x, 3))
        ))
    

class feedforward(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(config['emb_dim'],config['emb_dim']*4),
            GELU(),
            nn.Linear(config['emb_dim']*4,config['emb_dim']),

        )
    
    def forward(self,x):
        return self.layers(x)

In [5]:
class TransformerBlock(nn.Module):
    def __init__ (self,config):
        super().__init__()
        self.attn=multiheadv2(d_in=config['emb_dim'],d_out=config['emb_dim'],context_length=config['context_length'],dropout=config['drop_rate'],attention_head=config['n_heads'],boolbias=config['qkv_bias'])
        self.Layernorm1=LayerNorm(emb_dim=config['emb_dim'])
        self.Layernorm2=LayerNorm(emb_dim=config['emb_dim'])
        self.feedforw=feedforward(config=config)
        self.dropout=nn.Dropout(config['drop_rate'])
    
    def forward(self,x):
        ## attnetion block
        skip=x
        x=self.Layernorm1(x)
        x=self.attn(x)
        x=self.dropout(x)
        x=x+skip

        ## feed forward nn block
        skip=x
        x=self.Layernorm2(x)
        x=self.feedforw(x)
        x=self.dropout(x)
        x=x+skip

        return x


In [6]:
class GPT_2(nn.Module):
    def __init__ (self,cfg):
        super().__init__()
        self.token_emb=nn.Embedding(cfg['vocab_size'],cfg["emb_dim"])
        self.pos_emb=nn.Embedding(cfg['context_length'],cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    
    def forward(self,inputidx):
        batch_size,seq=inputidx.shape
        tokens=self.token_emb(inputidx)
        pos_embeds = self.pos_emb(torch.arange(seq, device=inputidx.device))
        x=tokens+pos_embeds
        x=self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    



In [8]:
torch.manual_seed(123)
model = GPT_2(GPT_CONFIG_124M)

out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.3613,  0.4223, -0.0711,  ...,  0.3483,  0.4661, -0.2838],
         [-0.1792, -0.5660, -0.9485,  ...,  0.0477,  0.5181, -0.3168],
         [ 0.7120,  0.0332,  0.1085,  ...,  0.1018, -0.4327, -0.2553],
         [-1.0076,  0.3418, -0.1190,  ...,  0.7195,  0.4023,  0.0532]],

        [[-0.2564,  0.0900,  0.0335,  ...,  0.2659,  0.4454, -0.6806],
         [ 0.1230,  0.3653, -0.2074,  ...,  0.7705,  0.2710,  0.2246],
         [ 1.0558,  1.0318, -0.2800,  ...,  0.6936,  0.3205, -0.3178],
         [-0.1565,  0.3926,  0.3288,  ...,  1.2630, -0.1858,  0.0388]]],
       grad_fn=<UnsafeViewBackward0>)


In [12]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

size=total_params*4/(1024*1024)
print(size,'mb size')

Total number of parameters: 163,009,536
621.83203125 mb size
